# Milestone 07: 待辦事項管理程式 - 詳細需求規格

## 📋 專案概述

本專案要求您設計並實作一個**命令列待辦事項管理系統 (CLI Todo Application)**，整合 Ch23-26 的檔案處理與資料持久化技術。這是一個實用的專案，將展示您對檔案 I/O、JSON、CSV、路徑管理的深度理解與實際應用能力。

### 🎯 整合技術要點
- **Ch23 檔案 I/O**：文字檔讀寫、with 語句、編碼處理
- **Ch24 JSON 處理**：資料序列化、反序列化、結構化儲存
- **Ch25 CSV 操作**：資料匯出、報表生成、表格處理
- **Ch26 路徑管理**：pathlib 使用、跨平台路徑、目錄管理

---

## 🏗️ 系統架構設計（System Architecture）

### 類別架構圖

```mermaid
classDiagram
    class Task {
        -id: str
        -title: str
        -description: str
        -priority: str
        -status: str
        -created_date: datetime
        -due_date: datetime
        -tags: List[str]
        +to_dict(): Dict
        +from_dict(data): Task
        +is_overdue(): bool
        +__str__(): str
        +__eq__(other): bool
    }

    class TodoManager {
        -tasks: List[Task]
        -data_dir: Path
        -data_file: Path
        -backup_file: Path
        +add_task(title, ...): Task
        +list_tasks(filter_by, sort_by): List[Task]
        +update_task(task_id, **kwargs): bool
        +delete_task(task_id): bool
        +mark_completed(task_id): bool
        +search_tasks(keyword): List[Task]
        +filter_by_priority(priority): List[Task]
        +filter_by_status(status): List[Task]
        +get_overdue_tasks(): List[Task]
        +get_statistics(): Dict
        +export_to_csv(filename): bool
        +save_to_file(): void
        +load_from_file(): void
        +create_backup(): void
    }

    class FileHandler {
        +save_json(data, filepath): bool
        +load_json(filepath): Dict
        +export_csv(tasks, filepath): bool
        +create_directory(path): void
        +backup_file(source, dest): bool
    }

    class TodoApp {
        -manager: TodoManager
        +run(): void
        +show_menu(): void
        +handle_add_task(): void
        +handle_list_tasks(): void
        +handle_update_task(): void
        +handle_delete_task(): void
        +handle_search(): void
        +handle_export(): void
        +handle_statistics(): void
    }

    TodoManager ||--o{ Task : manages
    TodoManager --> FileHandler : uses
    TodoApp --> TodoManager : uses
```

### 檔案系統結構

```
todo_app/
├── data/
│   ├── todos.json          # 主要資料檔案
│   ├── todos_backup.json   # 自動備份檔案
│   ├── archive/            # 封存目錄
│   │   └── archive_2025.json
│   └── exports/            # 匯出目錄
│       ├── tasks_export.csv
│       └── statistics_2025.csv
└── logs/
    └── error_log.txt       # 錯誤日誌（選做）
```

---

## 📋 功能需求詳細規格（Functional Requirements）

### 🔥 基本需求（必須完成 - 70分）

#### 1. Task 資料結構 (10分)

**核心屬性**：
```python
{
    "id": "unique_uuid",              # 唯一識別碼（UUID4 格式）
    "title": "string",                # 任務標題（1-100字元）
    "description": "string",          # 任務描述（選填，最多500字元）
    "priority": "High|Medium|Low",    # 優先級（預設 Medium）
    "status": "Pending|InProgress|Completed",  # 狀態（預設 Pending）
    "created_date": "2025-10-08T10:30:00",     # 建立時間（ISO 8601格式）
    "due_date": "2025-10-15T18:00:00",         # 截止時間（選填）
    "tags": ["work", "urgent"]        # 標籤列表（選填）
}
```

**必須實作的方法**：
```python
class Task:
    def __init__(self, title: str, description: str = "", 
                 priority: str = "Medium", due_date: str = None,
                 tags: List[str] = None):
        """初始化任務物件
        
        Args:
            title: 任務標題（必填）
            description: 任務描述
            priority: 優先級 (High/Medium/Low)
            due_date: 截止日期（ISO格式字串）
            tags: 標籤列表
        """
    
    def to_dict(self) -> Dict:
        """將 Task 物件轉換為字典（用於 JSON 序列化）
        
        Returns:
            Dict: 包含所有屬性的字典
        """
    
    @classmethod
    def from_dict(cls, data: Dict) -> 'Task':
        """從字典建立 Task 物件（用於 JSON 反序列化）
        
        Args:
            data: 包含任務資料的字典
        
        Returns:
            Task: 新建立的任務物件
        """
    
    def is_overdue(self) -> bool:
        """檢查任務是否逾期
        
        Returns:
            bool: 是否逾期（無截止日期返回 False）
        """
    
    def __str__(self) -> str:
        """回傳友善的字串表示
        
        格式範例：
        [High] 完成 Python 專案 (Pending) - 2025-10-15
        """
```

**評分標準**：
- 使用 UUID 生成唯一 ID (2分)
- 正確處理 datetime 物件與 ISO 格式轉換 (3分)
- 資料驗證（優先級、狀態必須為有效值） (3分)
- to_dict 與 from_dict 正確實作 (2分)

---

#### 2. 檔案持久化操作 (15分)

##### 2.1 儲存至 JSON 檔案 (8分)

```python
def save_to_file(self) -> bool:
    """儲存所有任務至 JSON 檔案
    
    步驟：
    1. 將所有 Task 物件轉換為字典列表
    2. 建立 metadata（最後修改時間、版本）
    3. 組合完整資料結構
    4. 先備份現有檔案（如果存在）
    5. 寫入新資料至檔案
    6. 使用 UTF-8 編碼、縮排格式化
    
    Returns:
        bool: 儲存是否成功
    
    Raises:
        IOError: 檔案寫入失敗
        PermissionError: 沒有寫入權限
    """
    try:
        # 1. 準備資料
        data = {
            "tasks": [task.to_dict() for task in self.tasks],
            "metadata": {
                "last_modified": datetime.now().isoformat(),
                "version": "1.0",
                "total_tasks": len(self.tasks)
            }
        }
        
        # 2. 建立備份
        if self.data_file.exists():
            shutil.copy(self.data_file, self.backup_file)
        
        # 3. 寫入檔案
        with open(self.data_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        
        return True
    except Exception as e:
        print(f"儲存失敗：{e}")
        return False
```

**評分標準**：
- 正確使用 json.dump() (2分)
- 使用 pathlib 處理路徑 (2分)
- UTF-8 編碼與 ensure_ascii=False (2分)
- 完整錯誤處理與備份機制 (2分)

##### 2.2 從 JSON 檔案讀取 (7分)

```python
def load_from_file(self) -> bool:
    """從 JSON 檔案載入任務
    
    步驟：
    1. 檢查檔案是否存在
    2. 讀取並解析 JSON
    3. 將字典資料轉換為 Task 物件
    4. 處理檔案損毀情況（嘗試從備份恢復）
    5. 處理空檔案或格式錯誤
    
    Returns:
        bool: 載入是否成功
    
    Raises:
        FileNotFoundError: 檔案不存在（建立新檔案）
        json.JSONDecodeError: JSON 格式錯誤（嘗試從備份恢復）
    """
    try:
        # 檔案不存在時建立空列表
        if not self.data_file.exists():
            self.tasks = []
            return True
        
        # 讀取檔案
        with open(self.data_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # 轉換為 Task 物件
        self.tasks = [Task.from_dict(task_data) 
                      for task_data in data.get("tasks", [])]
        return True
        
    except json.JSONDecodeError:
        # 嘗試從備份恢復
        print("主檔案損毀，嘗試從備份恢復...")
        return self._restore_from_backup()
    
    except Exception as e:
        print(f"載入失敗：{e}")
        return False
```

**評分標準**：
- 檔案存在性檢查 (2分)
- 正確使用 json.load() 與反序列化 (2分)
- 處理空檔案或損毀檔案 (2分)
- 備份恢復機制 (1分)

---

#### 3. 核心 CRUD 操作 (25分)

##### 3.1 新增任務 (8分)

```python
def add_task(self, title: str, description: str = "",
            priority: str = "Medium", due_date: str = None,
            tags: List[str] = None) -> Task:
    """新增待辦任務
    
    Args:
        title: 任務標題（必填，1-100字元）
        description: 任務描述（選填）
        priority: 優先級（High/Medium/Low）
        due_date: 截止日期（ISO格式字串）
        tags: 標籤列表
    
    Returns:
        Task: 新建立的任務物件
    
    Raises:
        ValueError: 標題為空或過長
        ValueError: 優先級無效
    """
    # 1. 驗證輸入
    if not title or len(title) > 100:
        raise ValueError("任務標題長度必須在 1-100 字元之間")
    
    if priority not in ["High", "Medium", "Low"]:
        raise ValueError(f"無效的優先級：{priority}")
    
    # 2. 建立 Task 物件
    task = Task(title, description, priority, due_date, tags)
    
    # 3. 加入任務清單
    self.tasks.append(task)
    
    # 4. 儲存至檔案
    self.save_to_file()
    
    print(f"✓ 任務已新增：{task.title}")
    return task
```

**評分標準**：
- 輸入驗證（標題、優先級） (2分)
- 正確建立 Task 物件 (2分)
- 加入任務清單 (2分)
- 自動儲存至檔案 (2分)

##### 3.2 列出任務 (6分)

```python
def list_tasks(self, filter_by: str = None, 
              sort_by: str = "created") -> List[Task]:
    """列出所有任務，支援過濾與排序
    
    Args:
        filter_by: 過濾條件（'pending', 'completed', 'high', etc.）
        sort_by: 排序依據（'created', 'due_date', 'priority'）
    
    Returns:
        List[Task]: 過濾並排序後的任務列表
    """
    # 1. 過濾
    filtered_tasks = self.tasks
    if filter_by:
        if filter_by.lower() in ["pending", "inprogress", "completed"]:
            filtered_tasks = [t for t in self.tasks 
                            if t.status.lower() == filter_by.lower()]
        elif filter_by.lower() in ["high", "medium", "low"]:
            filtered_tasks = [t for t in self.tasks 
                            if t.priority.lower() == filter_by.lower()]
    
    # 2. 排序
    if sort_by == "priority":
        priority_order = {"High": 0, "Medium": 1, "Low": 2}
        filtered_tasks.sort(key=lambda t: priority_order[t.priority])
    elif sort_by == "due_date":
        filtered_tasks.sort(key=lambda t: t.due_date or datetime.max)
    else:  # 預設按建立時間
        filtered_tasks.sort(key=lambda t: t.created_date)
    
    # 3. 格式化顯示
    if not filtered_tasks:
        print("目前沒有任務")
    else:
        for i, task in enumerate(filtered_tasks, 1):
            print(f"{i}. {task}")
    
    return filtered_tasks
```

**評分標準**：
- 過濾功能實作 (2分)
- 排序功能實作 (2分)
- 友善的格式化顯示 (2分)

##### 3.3 更新任務 (6分)

```python
def update_task(self, task_id: str, **kwargs) -> bool:
    """更新任務的指定欄位
    
    Args:
        task_id: 任務 ID
        **kwargs: 要更新的欄位（title, description, priority, status, etc.）
    
    Returns:
        bool: 更新是否成功
    
    Raises:
        ValueError: 任務不存在
    """
    # 1. 找到任務
    task = self._find_task_by_id(task_id)
    if not task:
        raise ValueError(f"任務 {task_id} 不存在")
    
    # 2. 更新欄位
    for key, value in kwargs.items():
        if hasattr(task, key):
            setattr(task, key, value)
    
    # 3. 儲存變更
    self.save_to_file()
    return True
```

##### 3.4 刪除任務 (3分)

```python
def delete_task(self, task_id: str) -> bool:
    """刪除指定任務
    
    Args:
        task_id: 任務 ID
    
    Returns:
        bool: 刪除是否成功
    """
    # 1. 找到任務
    task = self._find_task_by_id(task_id)
    if not task:
        print(f"任務 {task_id} 不存在")
        return False
    
    # 2. 確認刪除
    confirm = input(f"確定要刪除任務「{task.title}」嗎？(y/n): ")
    if confirm.lower() != 'y':
        print("已取消刪除")
        return False
    
    # 3. 移除任務並儲存
    self.tasks.remove(task)
    self.save_to_file()
    print(f"✓ 任務已刪除：{task.title}")
    return True
```

##### 3.5 標記完成 (2分)

```python
def mark_completed(self, task_id: str) -> bool:
    """標記任務為已完成
    
    Args:
        task_id: 任務 ID
    
    Returns:
        bool: 操作是否成功
    """
    return self.update_task(task_id, status="Completed")
```

---

#### 4. 搜尋與過濾功能 (10分)

##### 4.1 關鍵字搜尋 (5分)

```python
def search_tasks(self, keyword: str) -> List[Task]:
    """搜尋標題或描述中包含關鍵字的任務
    
    Args:
        keyword: 搜尋關鍵字（不區分大小寫）
    
    Returns:
        List[Task]: 符合條件的任務列表
    """
    keyword = keyword.lower()
    
    results = [
        task for task in self.tasks
        if keyword in task.title.lower() 
        or keyword in task.description.lower()
    ]
    
    print(f"找到 {len(results)} 個符合「{keyword}」的任務：")
    for task in results:
        print(f"  - {task}")
    
    return results
```

**評分標準**：
- 不區分大小寫搜尋 (2分)
- 搜尋標題與描述 (2分)
- 回傳結果列表與顯示 (1分)

##### 4.2 條件過濾 (5分)

```python
def filter_by_priority(self, priority: str) -> List[Task]:
    """依照優先級過濾任務
    
    Args:
        priority: 優先級（High/Medium/Low）
    
    Returns:
        List[Task]: 符合條件的任務
    """
    return [task for task in self.tasks 
            if task.priority == priority]

def filter_by_status(self, status: str) -> List[Task]:
    """依照狀態過濾任務
    
    Args:
        status: 狀態（Pending/InProgress/Completed）
    
    Returns:
        List[Task]: 符合條件的任務
    """
    return [task for task in self.tasks 
            if task.status == status]

def filter_by_tag(self, tag: str) -> List[Task]:
    """依照標籤過濾任務
    
    Args:
        tag: 標籤名稱
    
    Returns:
        List[Task]: 包含該標籤的任務
    """
    return [task for task in self.tasks 
            if tag in task.tags]
```

**評分標準**：
- 優先級過濾 (2分)
- 狀態過濾 (2分)
- 標籤過濾 (1分)

---

#### 5. 命令列介面 (10分)

```python
class TodoApp:
    """待辦事項應用程式主程式"""
    
    def __init__(self):
        self.manager = TodoManager()
        self.manager.load_from_file()
    
    def show_menu(self):
        """顯示主選單"""
        print("\n" + "="*50)
        print("📝 待辦事項管理系統 | Todo Manager")
        print("="*50)
        print("1. 新增任務")
        print("2. 列出所有任務")
        print("3. 搜尋任務")
        print("4. 更新任務")
        print("5. 刪除任務")
        print("6. 標記完成")
        print("7. 查看統計")
        print("8. 匯出 CSV")
        print("0. 退出")
        print("="*50)
    
    def run(self):
        """執行主程式迴圈"""
        while True:
            self.show_menu()
            choice = input("請選擇功能 (0-8): ").strip()
            
            if choice == "0":
                print("感謝使用！再見！")
                break
            elif choice == "1":
                self.handle_add_task()
            elif choice == "2":
                self.handle_list_tasks()
            elif choice == "3":
                self.handle_search()
            elif choice == "4":
                self.handle_update_task()
            elif choice == "5":
                self.handle_delete_task()
            elif choice == "6":
                self.handle_mark_completed()
            elif choice == "7":
                self.handle_statistics()
            elif choice == "8":
                self.handle_export()
            else:
                print("❌ 無效的選項，請重新選擇")
    
    def handle_add_task(self):
        """處理新增任務"""
        print("\n--- 新增任務 ---")
        title = input("標題（必填）: ").strip()
        description = input("描述（選填）: ").strip()
        priority = input("優先級 (High/Medium/Low，預設 Medium): ").strip() or "Medium"
        due_date = input("截止日期 (YYYY-MM-DD，選填): ").strip() or None
        tags_input = input("標籤（用逗號分隔，選填）: ").strip()
        tags = [tag.strip() for tag in tags_input.split(',')] if tags_input else None
        
        try:
            task = self.manager.add_task(title, description, priority, due_date, tags)
            print(f"\n✓ 任務已新增：{task}")
        except ValueError as e:
            print(f"\n❌ 新增失敗：{e}")
    
    def handle_list_tasks(self):
        """處理列出任務"""
        print("\n--- 列出任務 ---")
        filter_by = input("過濾條件（Pending/Completed/High/Medium/Low，留空顯示全部）: ").strip() or None
        sort_by = input("排序依據（created/due_date/priority，預設 created）: ").strip() or "created"
        
        self.manager.list_tasks(filter_by, sort_by)
    
    # ... 其他 handle_* 方法
```

**評分標準**：
- 清晰的選單顯示 (3分)
- 輸入驗證與錯誤處理 (3分)
- 迴圈與退出機制 (2分)
- 友善的使用者提示 (2分)

---

### 🚀 進階需求（選做 - 額外30分）

完整的進階需求請參閱 `README.md` 第三章節。

主要包含：
1. **截止日期管理** (8分)：逾期檢查、即將到期提醒
2. **資料匯出功能** (8分)：CSV 匯出、統計報表
3. **統計與報表** (5分)：任務統計、完成率計算
4. **標籤系統** (6分)：標籤過濾、標籤管理
5. **封存系統** (3分)：已完成任務封存

---

## 📋 完成檢核清單（Completion Checklist）

### 🔥 基本需求（70分）

#### Task 資料結構 (10分)
- [ ] UUID 唯一 ID 生成
- [ ] datetime 物件與 ISO 格式轉換
- [ ] 資料驗證（優先級、狀態）
- [ ] to_dict() 方法實作
- [ ] from_dict() 類別方法實作
- [ ] is_overdue() 逾期檢查
- [ ] `__str__()` 友善字串表示

#### 檔案持久化 (15分)
- [ ] save_to_file() JSON 儲存
- [ ] load_from_file() JSON 讀取
- [ ] pathlib 路徑處理
- [ ] UTF-8 編碼處理
- [ ] 自動備份機制
- [ ] 檔案損毀恢復
- [ ] 錯誤處理（FileNotFoundError, JSONDecodeError）

#### CRUD 操作 (25分)
- [ ] add_task() 新增任務
- [ ] list_tasks() 列出任務
- [ ] update_task() 更新任務
- [ ] delete_task() 刪除任務
- [ ] mark_completed() 標記完成
- [ ] 過濾功能（狀態、優先級）
- [ ] 排序功能（建立時間、截止日期、優先級）

#### 搜尋與過濾 (10分)
- [ ] search_tasks() 關鍵字搜尋
- [ ] filter_by_priority() 優先級過濾
- [ ] filter_by_status() 狀態過濾
- [ ] 不區分大小寫搜尋
- [ ] 搜尋標題與描述

#### 命令列介面 (10分)
- [ ] 清晰的選單顯示
- [ ] 主程式迴圈
- [ ] 輸入驗證
- [ ] 錯誤處理與提示
- [ ] 友善的使用者體驗

---

### 🚀 進階需求（30分）

請參閱 `README.md` 完整清單。

---

**🎯 完成此檢核清單即表示您已掌握檔案處理與資料持久化的核心技術！**

---

**文件版本**: v1.0  
**最後更新**: 2025-10-08  
**維護者**: iSpan Python 教學團隊